# 07 - TF-IDF + Logistic Regression (Semi-Supervised)

**Approach**: Train a linear classifier on TF-IDF features from lookup tables **and pseudo-labeled LinkedIn CVs**.

This approach uses a classic machine learning pipeline:
1. Load Gold (CSV) and Silver (Pseudo-labeled JSON) data.
2. Split Gold data into train/val set.
3. Train on **Gold Train + Silver**.
4. Evaluate on **Gold Val** (In-Distribution) and **Annotated JSON** (Real-World).

In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Add src to path
import sys
sys.path.append('../')
from src.data.loader import load_label_lists, load_inference_dataset, load_evaluation_dataset, balance_dataset

# Paths
DATA_DIR = Path('../data')
RESULTS_DIR = Path('./results')
MODELS_DIR = Path('../models')
RESULTS_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)

%matplotlib inline

In [2]:
# Load lookup tables (Gold)
dept_df, sen_df = load_label_lists(DATA_DIR)

# Load pseudo-labeled data (Silver)
silver_path = DATA_DIR / "processed/unannotated_pseudo_labeled.csv"
if silver_path.exists():
    silver_df = pd.read_csv(silver_path)
    # Silver DF has 'title' instead of 'text'
    dept_silver = silver_df[silver_df['dept_pseudo'].notna()][['title', 'dept_pseudo']].copy()
    dept_silver = dept_silver.rename(columns={'dept_pseudo': 'label', 'title': 'text'})
    
    sen_silver = silver_df[silver_df['sen_pseudo'].notna()][['title', 'sen_pseudo']].copy()
    sen_silver = sen_silver.rename(columns={'sen_pseudo': 'label', 'title': 'text'})
    
    print(f"🥈 Loaded {len(dept_silver)} dept and {len(sen_silver)} seniority silver labels")
else:
    dept_silver = pd.DataFrame(columns=['text', 'label'])
    sen_silver = pd.DataFrame(columns=['text', 'label'])
    print("⚠️ Silver data not found")

print(f"\n📊 Gold Department: {len(dept_df):,} examples")
print(f"📊 Gold Seniority:  {len(sen_df):,} examples")

Applying encoding fix...
Deduplicating department labels...
  Deduplication: 10145 -> 10145 (removed 0 duplicates)
  Capping: 10145 -> 2597 (max 500 per class)
Deduplicating seniority labels...
  Deduplication: 9428 -> 9428 (removed 0 duplicates)
  Capping: 9428 -> 2409 (max 500 per class)
🥈 Loaded 314 dept and 314 seniority silver labels

📊 Gold Department: 2,597 examples
📊 Gold Seniority:  2,409 examples


In [3]:
# Split Gold into train/val
dept_train_gold_texts, dept_val_texts, dept_train_gold_labels, dept_val_labels = train_test_split(
    dept_df['text'].tolist(),
    dept_df['label'].tolist(),
    test_size=0.2, random_state=42, stratify=dept_df['label']
)

sen_train_gold_texts, sen_val_texts, sen_train_gold_labels, sen_val_labels = train_test_split(
    sen_df['text'].tolist(),
    sen_df['label'].tolist(),
    test_size=0.2, random_state=42, stratify=sen_df['label']
)

print(f"Department: {len(dept_train_gold_texts)} gold + {len(dept_silver)} silver")
print(f"Seniority:  {len(sen_train_gold_texts)} gold + {len(sen_silver)} silver")

# Combine with Silver for training
dept_train_texts = dept_train_gold_texts + dept_silver['text'].tolist()
dept_train_labels = dept_train_gold_labels + dept_silver['label'].tolist()

sen_train_texts = sen_train_gold_texts + sen_silver['text'].tolist()
sen_train_labels = sen_train_gold_labels + sen_silver['label'].tolist()

# Apply Tier 1 Data Balancing
dept_train_df = pd.DataFrame({'text': dept_train_texts, 'label': dept_train_labels})
dept_train_df, dept_weights = balance_dataset(dept_train_df, min_samples=500, max_samples=2000, return_weights=True)
dept_train_texts = dept_train_df['text'].tolist()
dept_train_labels = dept_train_df['label'].tolist()

sen_train_df = pd.DataFrame({'text': sen_train_texts, 'label': sen_train_labels})
sen_train_df, sen_weights = balance_dataset(sen_train_df, min_samples=500, max_samples=2000, return_weights=True)
sen_train_texts = sen_train_df['text'].tolist()
sen_train_labels = sen_train_df['label'].tolist()

print(f"\nBalanced Department: {len(dept_train_texts)} examples")
print(f"Balanced Seniority:  {len(sen_train_texts)} examples")

Department: 2077 gold + 314 silver
Seniority:  1927 gold + 314 silver
Balancing: 2391 -> 5500 samples
  Class distribution: {'Sales': 500, 'Business Development': 500, 'Marketing': 500, 'Information Technology': 500, 'Project Management': 500, 'Consulting': 500, 'Administrative': 500, 'Other': 500, 'Purchasing': 500, 'Human Resources': 500, 'Customer Support': 500}
Balancing: 2241 -> 2500 samples
  Class distribution: {'Management': 500, 'Senior': 500, 'Lead': 500, 'Director': 500, 'Junior': 500}

Balanced Department: 5500 examples
Balanced Seniority:  2500 examples


## 2. Department Classifier: TF-IDF + Logistic Regression

In [4]:
# Create TF-IDF vectorizer for departments
dept_vectorizer = TfidfVectorizer(
    max_features=5000, 
    ngram_range=(1, 2),
    analyzer='word',
    lowercase=True,
    min_df=2,
    max_df=0.95
)

# Fit and transform training data
dept_X_train = dept_vectorizer.fit_transform(dept_train_texts)
dept_X_val = dept_vectorizer.transform(dept_val_texts)

print(f"Department TF-IDF features: {dept_X_train.shape[1]:,}")
print(f"Training matrix shape: {dept_X_train.shape}")

Department TF-IDF features: 3,012
Training matrix shape: (5500, 3012)


In [5]:
# Train Logistic Regression classifier
print("Training Department Logistic Regression...")
dept_logreg = LogisticRegression(
    max_iter=1000,
    C=1.0,
    solver='lbfgs',
    multi_class='multinomial',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

dept_logreg.fit(dept_X_train, dept_train_labels)

# Evaluate on validation set (In-distribution)
dept_val_preds = dept_logreg.predict(dept_X_val)
dept_id_acc = accuracy_score(dept_val_labels, dept_val_preds)
print(f"\n✅ Department Logistic Regression trained")
print(f"   Validation accuracy (In-Distribution): {dept_id_acc:.4f}")

Training Department Logistic Regression...

✅ Department Logistic Regression trained
   Validation accuracy (In-Distribution): 0.9327


## 3. Seniority Classifier: TF-IDF + Logistic Regression

In [6]:
sen_vectorizer = TfidfVectorizer(
    max_features=3000, 
    ngram_range=(1, 2),
    analyzer='word',
    lowercase=True,
    min_df=2,
    max_df=0.95
)

sen_X_train = sen_vectorizer.fit_transform(sen_train_texts)
sen_X_val = sen_vectorizer.transform(sen_val_texts)

print(f"Seniority TF-IDF features: {sen_X_train.shape[1]:,}")
print(f"Training matrix shape: {sen_X_train.shape}")

Seniority TF-IDF features: 1,550
Training matrix shape: (2500, 1550)


In [7]:
print("Training Seniority Logistic Regression...")
sen_logreg = LogisticRegression(
    max_iter=1000,
    C=1.0,
    solver='lbfgs',
    multi_class='multinomial',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

sen_logreg.fit(sen_X_train, sen_train_labels)

sen_val_preds = sen_logreg.predict(sen_X_val)
sen_id_acc = accuracy_score(sen_val_labels, sen_val_preds)
print(f"\n✅ Seniority Logistic Regression trained")
print(f"   Validation accuracy (In-Distribution): {sen_id_acc:.4f}")

Training Seniority Logistic Regression...

✅ Seniority Logistic Regression trained
   Validation accuracy (In-Distribution): 0.9523


## 4. Evaluation on Annotated Dataset (Real-World)

In [8]:
eval_df = load_evaluation_dataset(DATA_DIR)
eval_titles = eval_df['title'].tolist()
print(f"📊 Loaded {len(eval_df)} annotated CV positions for evaluation")

# Department Evaluation
eval_dept_X = dept_vectorizer.transform(eval_titles)
dept_predictions = dept_logreg.predict(eval_dept_X)
dept_true = eval_df['department'].tolist()
dept_accuracy = accuracy_score(dept_true, dept_predictions)
dept_precision, dept_recall, dept_f1, _ = precision_recall_fscore_support(dept_true, dept_predictions, average='macro', zero_division=0)
dept_weighted_f1 = precision_recall_fscore_support(dept_true, dept_predictions, average='weighted', zero_division=0)[2]
dept_report = classification_report(dept_true, dept_predictions, output_dict=True, zero_division=0)
dept_f1_scores = {label: metrics['f1-score'] for label, metrics in dept_report.items() if label not in ['accuracy', 'macro avg', 'weighted avg']}

# Seniority Evaluation
eval_sen_X = sen_vectorizer.transform(eval_titles)
sen_predictions = sen_logreg.predict(eval_sen_X)
sen_true = eval_df['seniority'].tolist()
sen_accuracy = accuracy_score(sen_true, sen_predictions)
sen_precision, sen_recall, sen_f1, _ = precision_recall_fscore_support(sen_true, sen_predictions, average='macro', zero_division=0)
sen_weighted_f1 = precision_recall_fscore_support(sen_true, sen_predictions, average='weighted', zero_division=0)[2]
sen_report = classification_report(sen_true, sen_predictions, output_dict=True, zero_division=0)
sen_f1_scores = {label: metrics['f1-score'] for label, metrics in sen_report.items() if label not in ['accuracy', 'macro avg', 'weighted avg']}

print("\n" + "="*60)
print("EVALUATION RESULTS (TF-IDF + Logistic Regression)")
print("="*60)
print(f"Department - In-Dist: {dept_id_acc:.4f}, Real-World: {dept_accuracy:.4f}")
print(f"Seniority  - In-Dist: {sen_id_acc:.4f}, Real-World: {sen_accuracy:.4f}")
print("="*60)

📊 Loaded 478 annotated CV positions for evaluation

EVALUATION RESULTS (TF-IDF + Logistic Regression)
Department - In-Dist: 0.9327, Real-World: 0.3891
Seniority  - In-Dist: 0.9523, Real-World: 0.4163


## 5. Save Results

In [9]:
results = {
    "approach": "TF-IDF + Logistic Regression",
    "department": {
        "in_distribution_accuracy": float(dept_id_acc),
        "real_world_accuracy": float(dept_accuracy),
        "real_world_precision": float(dept_precision),
        "real_world_recall": float(dept_recall),
        "real_world_f1_macro": float(dept_f1),
        "real_world_f1_weighted": float(dept_weighted_f1),
        "per_class_f1": {k: float(v) for k, v in dept_f1_scores.items()}
    },
    "seniority": {
        "in_distribution_accuracy": float(sen_id_acc),
        "real_world_accuracy": float(sen_accuracy),
        "real_world_precision": float(sen_precision),
        "real_world_recall": float(sen_recall),
        "real_world_f1_macro": float(sen_f1),
        "real_world_f1_weighted": float(sen_weighted_f1),
        "per_class_f1": {k: float(v) for k, v in sen_f1_scores.items()}
    },
    "metadata": {
        "training_samples": len(dept_train_texts) + len(sen_train_texts),
        "dept_train_samples": len(dept_train_texts),
        "dept_vocab_size": len(dept_vectorizer.vocabulary_),
        "sen_train_samples": len(sen_train_texts),
        "sen_vocab_size": len(sen_vectorizer.vocabulary_),
        "max_features": {"department": 5000, "seniority": 3000},
        "ngram_range": "(1, 2)",
        "C": 1.0
    },
    "timestamp": datetime.now().isoformat()
}

output_path = RESULTS_DIR / 'tfidf_logreg_results.json'
with open(output_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✅ Results saved to: {output_path}")

# Save models
with open(MODELS_DIR / 'tfidf_logreg_department.pkl', 'wb') as f:
    pickle.dump({'vectorizer': dept_vectorizer, 'classifier': dept_logreg}, f)
with open(MODELS_DIR / 'tfidf_logreg_seniority.pkl', 'wb') as f:
    pickle.dump({'vectorizer': sen_vectorizer, 'classifier': sen_logreg}, f)
print("✅ Models saved to models/ directory")


✅ Results saved to: results\tfidf_logreg_results.json
✅ Models saved to models/ directory
